***
# Car prices
***

1. Répertorier l'ensemble des marques de véhicules
2. Répertorier l'ensemble des modèles de véhicules
3. Répertorier l'ensemble des prix des véhicules
***

In [2]:
# Create folders and getting path
import os
cwd = os.getcwd()

# Loop to create folders
folder_names = ['Dataframe', 'Output', 'Data']

folders = {}
for folder_name in folder_names:
    folders[folder_name] = os.path.join(cwd, folder_name)

    if not os.path.exists(os.path.join(cwd, folder_name)):
        os.makedirs(os.path.join(cwd, folder_name))
        print(f'Le dossier « {folder_name} » a été créé')

    else:
        print(f'Le dossier « {folder_name} » est existant')


# Création des variables de path

# Déterminer si os est win ou linux pour définir les path
if os.name == 'nt':
    slash = '\\'
elif os.name == 'posix':
    slash = '/'

path_prog =     cwd + slash
path_data =     folders['Data'] + slash
path_df =       folders['Dataframe'] + slash
path_output =   folders['Output'] + slash

# Mettre \\ pour éviter les erreurs
path_dict = [path_prog, path_data, path_df, path_output]
for path in path_dict:
    path = path.replace('\\','\\\\')


# Détermination de l'année
print()
print(path_prog)
print(path_data)
print(path_df)
print(path_output)

Le dossier « Dataframe » est existant
Le dossier « Output » est existant
Le dossier « Data » est existant

c:\Users\Charles_tour\Documents\GitHub\car_sales_predictions\
c:\Users\Charles_tour\Documents\GitHub\car_sales_predictions\Data\
c:\Users\Charles_tour\Documents\GitHub\car_sales_predictions\Dataframe\
c:\Users\Charles_tour\Documents\GitHub\car_sales_predictions\Output\


In [26]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

link_original        = 'https://www.guideautoweb.com'
link_constructeur   = 'https://www.guideautoweb.com/constructeurs/'
response_api = requests.get(link_constructeur)
print(response_api.status_code)     # 200 is ok

200


In [7]:
# Get the data from api
data = response_api.text

# Convert Html to json
soup = BeautifulSoup(data, 'html.parser')

In [30]:
# Extract all marque and link
ul_element = soup.find('ul', id='brands-index-list')

data = {
    'marque' : [],
    'link_model' : []
}

# Extract information from <a> elements within the <ul> element
for a_element in ul_element.find_all('a'):
    a_text = a_element.get_text().strip()
    a_href = a_element.get('href')
    link_model = link_original + a_href

    data['marque'].append(a_text)
    data['link_model'].append(link_model)

df = pd.DataFrame(data)
df

,marque,link_model
0,Acura,https://www.guideautoweb.com/constructeurs/acura/
1,Alfa Romeo,https://www.guideautoweb.com/constructeurs/alf...
2,Allard,https://www.guideautoweb.com/constructeurs/all...
3,Aston Martin,https://www.guideautoweb.com/constructeurs/ast...
4,Audi,https://www.guideautoweb.com/constructeurs/audi/
...,...,...
73,Tesla,https://www.guideautoweb.com/constructeurs/tesla/
74,Toyota,https://www.guideautoweb.com/constructeurs/toy...
75,VinFast,https://www.guideautoweb.com/constructeurs/vin...
76,Volkswagen,https://www.guideautoweb.com/constructeurs/vol...


In [34]:
acura = df.iloc[0,1]

In [35]:
# Extract information from Acura page
acura_api = requests.get(acura)
acura_data = acura_api.text

# Convert Html to json
acura_soup = BeautifulSoup(acura_data, 'html.parser')
acura_soup

<!DOCTYPE html>

<html lang="fr">
<head>
<meta charset="utf-8"/>
<title>Acura - Essais, actualité, galeries photos et vidéos - Guide Auto</title>
<script>var lq=lq||[];(function(){jskit=document.createElement('script');jskit.src='/js/c/jquery.3,cookie,url,cssua,lazyload,lcm,lcm.nots,lcm.vid,pads,g17+main,g17+modernizr,1612291859';jskit.async=true;jskit.onload=function(){while(lq.length){try{lq.shift()()}catch(e){}}lq={push:function(a){a()}}};document.head.appendChild(jskit)})();</script>
<script>lq.push(function(){jQuery(document).on('click','a[href]:not(.no-popup)',function(){if(/^https?:\/\/(?!(?:[^.]+\.)*www.guideautoweb.com)/.test(jQuery(this).attr('href')))jQuery(this).attr('target','_blank')})});</script>
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src='https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertB

In [ ]:
# Extract all marque and link
acura_ul_element = acura_soup.find('ul', id='brands-index-list')

acura_data = {
    'marque' : [],
    'link_model' : []
}

# Extract information from <a> elements within the <ul> element
for element in acura_ul_element.find_all('a'):
    a_text = element.get_text().strip()
    a_href = element.get('href')
    print()
    # link_model = link_original + a_href

    # data['marque'].append(a_text)
    # data['link_model'].append(link_model)

# df = pd.DataFrame(data)
# df

In [ ]:
link_original

In [ ]:
# Extract list of all models
# Extract list of web links to the models

In [ ]:
# Extract list of 'model en production'
# Extract web link to those models

In [ ]:
# Extract price and fuel consomption for every model